In [3]:
import pandas as pd

In [4]:
# I will try to improve the model using an evenly distributed  technique
df = pd.read_csv("New_London.csv",encoding= 'unicode_escape')
df.head()

,Unnamed: 0,Property Name,Review Rating,Review Title,Review Text,Location Of The Reviewer,Date Of Review
0,0,Apex London Wall Hotel,5.0,Ottima qualitÃ prezzo,Siamo stati a Londra per un week end ed abbiam...,"Casale Monferrato, Italy",10/20/2012
1,1,Corinthia Hotel London,5.0,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,"Savannah, Georgia",3/23/2016
2,2,The Savoy,5.0,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,London,7/30/2013
3,3,Rhodes Hotel,4.0,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,"Maui, Hawaii",06/02/2012
4,4,The Savoy,5.0,Perfection,Form the moment we arrived until we left we ex...,"London, United Kingdom",11/24/2017


In [5]:
import random

class Sentiment:
    positive = "Positive"
    negative = "Negative"
    neural = "Neural"
    


class Review:
    def __init__(self, text, rating):
        self.text = text
        self.rating = rating
        self.sentiment = self.get_rating()
        
    def get_rating(self):
        if self.rating > 4:
            return Sentiment.positive
        elif self.rating >= 3 and self.rating <= 4 :
            return Sentiment.neural
        else:
            return Sentiment.negative
        
class Review_text:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
        
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.negative, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.positive, self.reviews))
        positive_shrink = positive[:len(negative)]
        self.reviews = negative + positive_shrink
        random.shuffle(self.reviews)        

In [6]:
reviews = []
for index,item in df.iterrows():
    reviews.append(Review(item["Review Text"], item["Review Rating"]))
    
# Cheching some data
reviews[0].rating

5.0

In [7]:
# Split the data to train and test the model
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_review = Review_text(training)
test_review = Review_text(test)



In [8]:
print(len(training))
len((test))

15662


7715

In [9]:
# Developed an evenly distribute between positives and negatives comments
train_review.evenly_distribute()

train_x = train_review.get_text()
train_y = train_review.get_sentiment()

test_review.evenly_distribute()

test_x = test_review.get_text()
test_y = test_review.get_sentiment()

print(train_y.count(Sentiment.positive))
train_y.count((Sentiment.negative))

722


722

In [10]:
# Converting the text to numeric type
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = vectorizer = TfidfVectorizer() 

# fit and transform 

train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x_vectors[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [11]:
# Train the SVM model

from sklearn import svm

clf_svm = svm.SVC(kernel="linear")

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[1:10])

array(['Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Positive', 'Positive', 'Negative', 'Negative'], dtype='<U8')

In [12]:
# Evaluation
## Accuracy
print(clf_svm.score(train_x_vectors, train_y))

0.9944598337950139


In [14]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf_svm, train_x_vectors, train_y, cv=10, scoring="accuracy")
scores

array([0.92413793, 0.92413793, 0.93103448, 0.95172414, 0.9375    ,
       0.93055556, 0.94444444, 0.89583333, 0.90972222, 0.86805556])

In [111]:
## The model is overfitting, I will try to impement a GridSearch in order to tune the parameter C
## Optimaze the model
import numpy as np
from sklearn.model_selection import GridSearchCV

param={'kernel':('linear', 'poly'),
      'C': np.arange(0.1,10,0.1),
      }

SVM = GridSearchCV(estimator=clf_svm,
            param_grid=param,
            scoring='accuracy',
            cv=10
           )

In [112]:
SVM.fit(train_x_vectors, train_y)

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0....
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]),
                         'kernel': ('linear', 'poly')},
             pre_dispatch='2*n_jobs', refit=True, ret

In [113]:
## Creating a table with the results
cv_results = pd.DataFrame(SVM.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,4.037735,1.171166,0.457087,0.246844,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.910345,0.868966,0.903448,0.910345,0.895833,0.861111,0.881944,0.847222,0.881944,0.854167,0.881533,0.022015,189
1,3.449026,0.740369,0.408758,0.153159,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.496552,0.496552,0.710345,0.772414,0.506944,0.520833,0.541667,0.520833,0.534722,0.506944,0.560781,0.092435,198
2,3.837224,1.286547,0.424434,0.176156,0.2,linear,"{'C': 0.2, 'kernel': 'linear'}",0.882759,0.882759,0.903448,0.917241,0.902778,0.881944,0.888889,0.861111,0.881944,0.875000,0.887787,0.015273,183
3,5.224795,0.940100,0.538054,0.244624,0.2,poly,"{'C': 0.2, 'kernel': 'poly'}",0.551724,0.551724,0.537931,0.579310,0.555556,0.562500,0.541667,0.597222,0.590278,0.555556,0.562347,0.019042,197
4,2.318528,0.183349,0.241222,0.014456,0.3,linear,"{'C': 0.30000000000000004, 'kernel': 'linear'}",0.889655,0.896552,0.903448,0.951724,0.909722,0.881944,0.916667,0.868056,0.868056,0.875000,0.896082,0.024529,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2.495012,0.181740,0.246447,0.011060,9.7,poly,"{'C': 9.700000000000001, 'kernel': 'poly'}",0.903448,0.862069,0.903448,0.937931,0.881944,0.888889,0.930556,0.875000,0.888889,0.840278,0.891245,0.027988,98
194,2.108073,0.053198,0.156156,0.011916,9.8,linear,"{'C': 9.8, 'kernel': 'linear'}",0.931034,0.924138,0.951724,0.972414,0.951389,0.909722,0.951389,0.888889,0.909722,0.875000,0.926542,0.029393,10
195,2.500541,0.088959,0.258676,0.016162,9.8,poly,"{'C': 9.8, 'kernel': 'poly'}",0.903448,0.862069,0.903448,0.937931,0.881944,0.888889,0.930556,0.875000,0.888889,0.840278,0.891245,0.027988,98
196,2.186016,0.102297,0.161189,0.020328,9.9,linear,"{'C': 9.9, 'kernel': 'linear'}",0.931034,0.924138,0.951724,0.972414,0.951389,0.909722,0.951389,0.888889,0.909722,0.875000,0.926542,0.029393,10


In [128]:
svm_model_2 = svm.SVC(C=0.2, kernel="linear")

In [129]:
svm_model_2.fit(train_x_vectors, train_y)

SVC(C=0.2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [130]:
y_pred = svm_model_2.predict(train_x_vectors)

In [131]:
## Accuracy
from sklearn import metrics
print("The Accuracy of the model's training set is ", metrics.accuracy_score(y_pred,train_y))

The Accuracy of the model's training set is  0.9182825484764543


In [132]:
scores = cross_val_score(svm_model_2, train_x_vectors, train_y, cv=10, scoring="accuracy")
scores

array([0.88275862, 0.88275862, 0.90344828, 0.91724138, 0.90277778,
       0.88194444, 0.88888889, 0.86111111, 0.88194444, 0.875     ])

In [137]:
# F1 Scores 
from sklearn.metrics import f1_score

print(f1_score(train_y, svm_model_2.predict(train_x_vectors), average=None, labels=(Sentiment.positive, Sentiment.negative)))


[0.9119403  0.92377261]


In [139]:
## Let's now implement the model to the test set
y_pred = svm_model_2.predict(test_x_vectors)

In [140]:
print("The Accuracy of the model's test set is ", metrics.accuracy_score(y_pred,test_y))

The Accuracy of the model's test set is  0.9089635854341737


In [143]:
## Compute test set F1 scores
print(f1_score(test_y, svm_model_2.predict(test_x_vectors), average=None, labels=(Sentiment.positive, Sentiment.negative)))

[0.90284006 0.914361  ]


In [144]:
# Quality analysis 

test_set = ["Great place", "The most beautiful room", "It was a very bad hotel", "I don not recommend this place"]
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['Positive', 'Positive', 'Negative', 'Negative'], dtype='<U8')

In [145]:
## Save the model

In [146]:
import pickle 

In [149]:
with open("sentiment_classifier.pkl", "wb") as f:
     pickle.dump(clf_svm,f)

In [150]:
## Load the model
with open("sentiment_classifier.pkl", "rb") as f:
    loaded_clf = pickle.load(f)

In [151]:
# For instance
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

Em frente ao Albert e Victoria museu fica o Rembrandt. TambÃ©m facilmente acessÃ­vel de Ã´nibus( ponto na porta) e metrÃ´( do lado da South Kensington) . Hotel limpo, com cama confortÃ¡vel e com bom serviÃ§o. Cafe da manha excelente. Tem ainda um plus que Ã© um celular que vc pode usar internet e fazer ligaÃ§Ãµes dentro da cidade - coisa que nunca tinha visto em outros hotÃ©is. NÃ£o tivemos nenhum problema e a estadia foi muito agradÃ¡vel..e atÃ© barata se considerarmos outros hotÃ©is com outros preÃ§os em Londres.recomendo. O Lounge bar 1660 Ã© muito bom.


array(['Positive'], dtype='<U8')